<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/mandarin_frequency_analysis/pinyin_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using this to provide example word for the heisig deck

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# lifted from here https://stackoverflow.com/questions/8200349/convert-numbered-pinyin-to-pinyin-with-tone-marks
pinyinToneMarks = {
    'a': 'āáǎà', 'e': 'ēéěè', 'i': 'īíǐì',
    'o': 'ōóǒò', 'u': 'ūúǔù', 'ü': 'ǖǘǚǜ',
    'A': 'ĀÁǍÀ', 'E': 'ĒÉĚÈ', 'I': 'ĪÍǏÌ',
    'O': 'ŌÓǑÒ', 'U': 'ŪÚǓÙ', 'Ü': 'ǕǗǙǛ'
}

def convertPinyinCallback(m):
    tone=int(m.group(3))%5
    r=m.group(1).replace('v', 'ü').replace('V', 'Ü')
    # for multple vowels, use first one if it is a/e/o, otherwise use second one
    pos=0
    if len(r)>1 and not r[0] in 'aeoAEO':
        pos=1
    if tone != 0:
        r=r[0:pos]+pinyinToneMarks[r[pos]][tone-1]+r[pos+1:]
    return r+m.group(2)

def convertPinyin(s):
    return re.sub(r'([aeiouüvÜ]{1,3})(n?g?r?)([012345])', convertPinyinCallback, s, flags=re.IGNORECASE)

print(convertPinyin('Ni3 hao3 ma0?'))

Nǐ hǎo ma?


In [ ]:
file_to_edit = "https://raw.githubusercontent.com/Tclack88/misc-analysis/main/mandarin_frequency_analysis/Heisig1_to_edit.csv"
headers =  ["Hanzi", "Meaning", "Notes", "Words"]
old = pd.read_csv(file_to_edit, index_col=False, names=headers)
old.head()

,Hanzi,Meaning,Notes,Words
0,一,one,yī,NaN
1,二,two,èr,NaN
2,三,three,sān,NaN
3,四,four,sì,NaN
4,五,five,wǔ,NaN


In [ ]:
# OLD there's an issue sometimes with getting characters, this one has been replaced and changed for one that increments the page number and finds the next bunch until enough have been accumulated

# def get_example_words(char):
#   try:
#     url_start = "https://learn-chinese-words.com/dictionary.php?input="
#     source = requests.get(url_start+char).text
#     soup = BeautifulSoup(source, 'html.parser')
#     results = soup.find('table', {'class':'results'})
#     df = pd.read_html(str(results))[0].fillna('') #str for convertPinyin, NaN=>''
#     df[1] = df[1].apply(convertPinyin)
#     df[2] = df[2].apply(convertPinyin)
#     ch = list(df[0])
#     py = list(df[1])
#     en = list(df[2])
#     samples = '<br>'.join([' '.join(sample) for sample in list(zip(ch,py,en))])
#     return samples
#   except Exception as e:
#     print(f'problem withi {char}:\n{e}')
#     return ''


# # test_slice['Words'] = test_slice['Hanzi'].apply(get_example_words)

In [20]:
def get_example_words(char):
  samples = ''
  blank = 3*'&nbsp;'
  try:
    df = pd.DataFrame(columns=[0,1,2])
    # print(df)
    enough = 0
    n=1
    while not enough:
      url_start = f"https://learn-chinese-words.com/dictionary.php?page={n}&input="
      source = requests.get(url_start+char).text
      # print(url_start+char)
      soup = BeautifulSoup(source, 'html.parser')
      results = soup.find('table', {'class':'results'})
      small_df = pd.read_html(str(results))[0].fillna('') #str for convertPinyin, NaN=>''
      df = pd.concat([df,small_df])
      # print(df)

      df[1] = df[1].apply(convertPinyin)
      df[2] = df[2].apply(convertPinyin)
      df = df[df[0].str.contains(char)]
      if df.shape[0] >= 10: #if there are at least 5 examples grabbed, stop. otherwise check next page
        enough = 1
      else:
        n+=1
    ch = list(df[0])
    py = list(df[1])
    en = list(df[2])
    # print(df)
    samples += '<br>'.join([blank.join(sample) for sample in list(zip(ch,py,en))])
    # print(samples)
  except Exception as e:
    print(f'problem with {char}:\n{e}')
  return samples

# prob = '型'
# prob = '据'
# out = get_example_words(prob)
# print(out)

time: 2.63 ms (started: 2023-04-09 10:22:59 +00:00)


Test done. Time for actual conversion

In [3]:
!pip install ipython-autotime

%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
time: 577 µs (started: 2023-04-09 08:01:34 +00:00)


In [ ]:
# old['Words'] = old['Hanzi'].apply(get_example_words)

problem withi 的<br>(2):
No tables found matching pattern '.+'
problem withi 会 (2):
No tables found matching pattern '.+'
problem withi 周 (2):
No tables found matching pattern '.+'
problem withi 带 (2):
No tables found matching pattern '.+'


KeyboardInterrupt: ignored

time: 40min 36s (started: 2021-12-05 03:44:46 +00:00)


In [ ]:
old.to_csv('heisig1_with_examples',index=False)

In [ ]:
# temp
headers =  ["Hanzi", "Meaning", "Notes", "Words"]
editme = pd.read_csv('heisig1_to_edit.csv', index_col=False, names=headers)
# editme.Words = old.Words
editme

,Hanzi,Meaning,Notes,Words
0,一,one,yī,NaN
1,二,two,èr,NaN
2,三,three,sān,NaN
3,四,four,sì,NaN
4,五,five,wǔ,NaN
...,...,...,...,...
1466,亦,likewise,yì,NaN
1467,极,extreme,jí,NaN
1468,杂,miscellaneous,zá,NaN
1469,曰,sayeth,yuē,NaN


time: 44.7 ms (started: 2021-12-05 04:27:42 +00:00)


In [ ]:
editme['Words'] = editme['Hanzi'].apply(get_example_words)

problem withi 的<br>(2):
No tables found matching pattern '.+'
problem withi 会 (2):
No tables found matching pattern '.+'
problem withi 周 (2):
No tables found matching pattern '.+'
problem withi 带 (2):
No tables found matching pattern '.+'
time: 46min 19s (started: 2021-12-05 04:28:16 +00:00)


In [ ]:
editme.Words.str.replace('<br>','\n',regex=True)
def split_newlines(text):
  return '\n'.join([x.strip() for x in text.split('<br>')])
# df1 = pd.DataFrame({'Words':[x.strip() for x in editme.split('<br>')]})
editme.Words = editme.Words.apply(lambda x : x.replace('<br>', '\\n'))
# editme.Words.apply(split_newlines)

time: 18.5 ms (started: 2021-12-05 06:07:36 +00:00)


In [ ]:
editme.head()

,Hanzi,Meaning,Notes,Words
0,一,one,yī,一-yī-all\n一个-yī gè-a\n一些-yī xiē-a few\n第一-dì y...
1,二,two,èr,政府-zhèng fǔ-government\n二-èr-2\n二十-èr shí-20\n...
2,三,three,sān,三-sān-3\n三角形-sān jiǎo xíng-triangle\n第三-dì sān...
3,四,four,sì,四-sì-4\n四人帮-sì rén bāng-Truncated - click thro...
4,五,five,wǔ,应该-yīng gāi-should\n反应-fǎn yìng-reply\n应用-yìng...


time: 23.9 ms (started: 2021-12-05 06:07:58 +00:00)


In [ ]:
editme.to_csv('heisig1_with_examples',index=False, header=False)

time: 31.7 ms (started: 2021-12-05 05:18:34 +00:00)


In [ ]:
old.tail()

,Hanzi,Meaning,Notes,Words
1466,亦,likewise,yì,温度-wēn dù-temperature<br>速度-sù dù-rate<br>程度-c...
1467,极,extreme,jí,积极-jī jí-actively<br>极大-jí dà-maximum<br>极其-jí...
1468,杂,miscellaneous,zá,复杂-fù zá-complex<br>杂志-zá zhì-magazine<br>杂质-z...
1469,曰,sayeth,yuē,曰-yuē-to state<br>子曰-zǐ yuē-Confucius says:<br...
1470,乱,chaos,luàn,混乱-hùn luàn hún luàn-chaos<br>乱-luàn-riot<br>乱...


time: 20.2 ms (started: 2021-12-03 02:45:48 +00:00)


heisig 2

In [29]:
h2_dat = 'https://raw.githubusercontent.com/Tclack88/misc-analysis/main/mandarin_frequency_analysis/heisig2.csv'
h2_df = pd.read_csv(h2_dat, sep=',').fillna('')[['Hanzi','Meaning','Notes']]
h2_df.head()

,Hanzi,Meaning,Notes
0,厉,stern,lì
1,状,state of affairs,zhuàng
2,逃,escape,táo
3,逊,modest,xùn
4,略,abbreviation,lüè


time: 140 ms (started: 2023-04-09 10:27:17 +00:00)


In [30]:
h2_df['Words'] = h2_df['Hanzi'].apply(get_example_words)

problem with 矗:
No tables found matching pattern '.+'
problem with 嘿:
No tables found matching pattern '.+'
problem with 炯:
No tables found matching pattern '.+'
problem with 寞:
No tables found matching pattern '.+'
problem with 捎:
No tables found matching pattern '.+'
problem with 睬:
No tables found matching pattern '.+'
problem with 榕:
No tables found matching pattern '.+'
problem with 锹:
No tables found matching pattern '.+'
problem with 憋:
No tables found matching pattern '.+'
problem with 袱:
No tables found matching pattern '.+'
problem with 屉:
No tables found matching pattern '.+'
problem with 呻:
No tables found matching pattern '.+'
problem with 唉:
No tables found matching pattern '.+'
problem with 挎:
No tables found matching pattern '.+'
problem with 哟:
No tables found matching pattern '.+'
problem with 尴:
No tables found matching pattern '.+'
problem with 尬:
No tables found matching pattern '.+'
problem with 馒:
No tables found matching pattern '.+'
problem with 哎:
No tables fo

In [28]:
h2_df

'厉害&nbsp;&nbsp;&nbsp;lì hai&nbsp;&nbsp;&nbsp;violent<br>严厉&nbsp;&nbsp;&nbsp;yán lì&nbsp;&nbsp;&nbsp;serious<br>厉声&nbsp;&nbsp;&nbsp;lì shēng&nbsp;&nbsp;&nbsp;stern voice<br>厉&nbsp;&nbsp;&nbsp;lì&nbsp;&nbsp;&nbsp;strict<br>加厉&nbsp;&nbsp;&nbsp;jiā lì&nbsp;&nbsp;&nbsp;aggravation<br>厉色&nbsp;&nbsp;&nbsp;lì sè&nbsp;&nbsp;&nbsp;<br>变本加厉&nbsp;&nbsp;&nbsp;biàn běn jiā lì&nbsp;&nbsp;&nbsp;to intensify<br>厉行节约&nbsp;&nbsp;&nbsp;lì xíng jié yuē&nbsp;&nbsp;&nbsp;<br>粗厉&nbsp;&nbsp;&nbsp;cū lì&nbsp;&nbsp;&nbsp;abrasive<br>再接再厉&nbsp;&nbsp;&nbsp;zài jiē zài lì&nbsp;&nbsp;&nbsp;to persist<br>厉行&nbsp;&nbsp;&nbsp;lì xíng&nbsp;&nbsp;&nbsp;<br>凄厉&nbsp;&nbsp;&nbsp;qī lì&nbsp;&nbsp;&nbsp;sad and shrill<br>凌厉&nbsp;&nbsp;&nbsp;líng lì&nbsp;&nbsp;&nbsp;'

time: 6.2 ms (started: 2023-04-09 10:26:58 +00:00)


In [31]:
h2_df.to_csv('heisig2_with_examples',index=False)

time: 51.3 ms (started: 2023-04-09 11:48:11 +00:00)


Improved hunt with more common words

In [ ]:
test_slice = old.sample(n=25)
for ch in test_slice.Hanzi:
  print(ch)
  #I noticed the search bar populates with the hex code of the character
  #ex. 四 -> ox56db -> /5/56DB (the first number is repeated)
  tail = str(hex(ord(ch))).upper()[2:] 
  url = f'http://chinese-characters.org/contained/{tail[0]}/{tail}.html'
  print(url)
  try:
    a = pd.read_html(url, header=0)
  except Exception as e:
    print(e)
    print(f'{ch} NOT FOUND')
    continue
  print(len(a))
  good_tables = []
  for table in a:
    clean_table = table.dropna(axis=0,how='all').dropna(axis=1, how='all')
    if clean_table.columns.to_list() == ['Simp', 'Pinyin', 'English']:
      good_tables.append(clean_table)
  print(len(good_tables))
  #last = a[-1].dropna(axis=0,how='all').dropna(axis=1, how='all')
  #twotolast = a[-2].dropna(axis=0,how='all').dropna(axis=1, how='all')
  #print(last.columns)
  #print(last.head(3))
  #print(twotolast.columns)
  #print(twotolast.head(3))

价
http://chinese-characters.org/contained/4/4EF7.html
5
0
堡
http://chinese-characters.org/contained/5/5821.html
7
2
无
http://chinese-characters.org/contained/6/65E0.html
5
0
讨
http://chinese-characters.org/contained/8/8BA8.html
HTTP Error 404: Not Found
讨 NOT FOUND
板
http://chinese-characters.org/contained/6/677F.html
7
2
迈
http://chinese-characters.org/contained/8/8FC8.html
HTTP Error 404: Not Found
迈 NOT FOUND
器
http://chinese-characters.org/contained/5/5668.html
7
2
故
http://chinese-characters.org/contained/6/6545.html
8
2
温
http://chinese-characters.org/contained/6/6E29.html
6
0
袭
http://chinese-characters.org/contained/8/88AD.html
HTTP Error 404: Not Found
袭 NOT FOUND
沼
http://chinese-characters.org/contained/6/6CBC.html
8
2
区
http://chinese-characters.org/contained/5/533A.html
HTTP Error 404: Not Found
区 NOT FOUND
革
http://chinese-characters.org/contained/9/9769.html
10
2
颗
http://chinese-characters.org/contained/9/9897.html
HTTP Error 404: Not Found
颗 NOT FOUND
激
http://chinese-

In [ ]:
a = pd.read_html('http://chinese-characters.org/contained/5/53E3.html', header=0)
a

[                                                  Simp  ... Words
 0                                                  NaN  ...   NaN
 1                                                    𧥣  ...     0
 2                                                    𢒫  ...     0
 3                                                    𨙫  ...     0
 4                                                    害  ...    13
 ..                                                 ...  ...   ...
 689                                                长江口  ...   NaN
 690                                                喷火口  ...   NaN
 691                                                注入口  ...   NaN
 692                                                NaN  ...   NaN
 693  Home  Pinyin index  Radical index  Links  Abou...  ...   NaN
 
 [694 rows x 15 columns], Empty DataFrame
 Columns: [Home  Pinyin index  Radical index  Links  About  Search for a character:  Please spread the word!  var addthis_config = {data_track_addressba

time: 2.87 s (started: 2021-12-02 05:29:04 +00:00)


In [ ]:
a[7].dropna(axis=0,how='all').dropna(axis=1, how='all')

,Simp,Trad,Pinyin,English,Semantic,Phonetic,Words
1,吡,吡,BI3,blame,NaN,NaN,2.0
2,呋,呋,FU1,NaN,NaN,NaN,1.0
3,咣,咣,GUANG1,NaN,NaN,NaN,0.0
4,啉,啉,LAN2,slow,NaN,NaN,2.0
5,嘭,嘭,PENG1,drive_away,NaN,NaN,0.0
...,...,...,...,...,...,...,...
558,哫,哫,zu2,cajole,NaN,NaN,0.0
559,嘴,嘴,zui3,mouth,NaN,NaN,13.0
560,噿,噿,zui3,NaN,NaN,NaN,0.0
561,噂,噂,zun3,meet,口,尊,0.0


time: 34.6 ms (started: 2021-12-02 05:30:10 +00:00)


Side edit to add changes since my deck has now changed

In [ ]:
headers =  ["Hanzi", "Meaning", "Notes", "Words"]
base_data = pd.read_csv('heisig1_to_edit.csv', index_col=False, names=headers) #uploaded file
examples_data = pd.read_csv('heisig1_with_examples.csv', index_col=False, names=headers) #uploaded file
print(base_data.shape)
base_data.head()

(1471, 4)


,Hanzi,Meaning,Notes,Words
0,一,one,yī,NaN
1,二,two,èr,NaN
2,三,three,sān,NaN
3,四,four,sì,NaN
4,五,five,wǔ,NaN


In [ ]:
print(examples_data.shape)
examples_data.head()

(1471, 4)


,Hanzi,Meaning,Notes,Words
0,一,one,yī,一-yī-all<br>一个-yī gè-a<br>一些-yī xiē-a few<br>第...
1,二,two,èr,政府-zhèng fǔ-government<br>二-èr-2<br>二十-èr shí-...
2,三,three,sān,三-sān-3<br>三角形-sān jiǎo xíng-triangle<br>第三-dì...
3,四,four,sì,四-sì-4<br>四人帮-sì rén bāng-Truncated - click th...
4,五,five,wǔ,应该-yīng gāi-should<br>反应-fǎn yìng-reply<br>应用-...


In [ ]:
base_data.Words = examples_data.Words
base_data.head()

,Hanzi,Meaning,Notes,Words
0,一,one,yī,一-yī-all<br>一个-yī gè-a<br>一些-yī xiē-a few<br>第...
1,二,two,èr,政府-zhèng fǔ-government<br>二-èr-2<br>二十-èr shí-...
2,三,three,sān,三-sān-3<br>三角形-sān jiǎo xíng-triangle<br>第三-dì...
3,四,four,sì,四-sì-4<br>四人帮-sì rén bāng-Truncated - click th...
4,五,five,wǔ,应该-yīng gāi-should<br>反应-fǎn yìng-reply<br>应用-...


In [ ]:
def dash2space(x):
  #return x.replace('-', 3*'&nbsp;')
  return re.sub('-', 3*'&nbsp;', x)
base_data.Words = base_data.Words.fillna('').apply(dash2space)
base_data.head()

,Hanzi,Meaning,Notes,Words
0,一,one,yī,一&nbsp;&nbsp;&nbsp;yī&nbsp;&nbsp;&nbsp;all<br>...
1,二,two,èr,政府&nbsp;&nbsp;&nbsp;zhèng fǔ&nbsp;&nbsp;&nbsp;...
2,三,three,sān,三&nbsp;&nbsp;&nbsp;sān&nbsp;&nbsp;&nbsp;3<br>三...
3,四,four,sì,四&nbsp;&nbsp;&nbsp;sì&nbsp;&nbsp;&nbsp;4<br>四人...
4,五,five,wǔ,应该&nbsp;&nbsp;&nbsp;yīng gāi&nbsp;&nbsp;&nbsp;...


In [ ]:
base_data.to_csv('heisig1', header=False, index=False)

In [ ]:
alt_source_in = "https://raw.githubusercontent.com/agj/3000-traditional-hanzi/master/data/external/heisig-simplified.txt"
# a[['first','second']] = a["bad"].str.split(" ", 1, expand=True)
a = pd.read_csv(alt_source_in, delim_whitespace=True,skiprows=1).fillna('')
# a.columns=['bad']
# a.reset_index
a.head()

,#,Index,-,Character,-.1,Keyword
0,0001,一,one,,,
1,0002,二,two,,,
2,0003,三,three,,,
3,0004,四,four,,,
4,0005,五,five,,,


In [ ]:
# b[['A','B']] = a[['Index'], ''.join(['-', 'Character', '-.1', 'Keyword'])]
# b
a['last'] = a[['-', 'Character', '-.1', 'Keyword']].agg(' '.join, axis=1)
b = a[['Index','last']]
b.columns = ['Hanzi', 'Meaning']
b

,Hanzi,Meaning
0,一,one
1,二,two
2,三,three
3,四,four
4,五,five
...,...,...
3048,踊,eagerly
3049,錶,watch
3050,锺,draw together
3051,闺,small door


In [ ]:
old[old.Meaning == '']

,Hanzi,Meaning,Notes,Words


In [ ]:
a

bad    first    second
0001 一            one      one      None
0002 二            two      two      None
0003 三          three    three      None
0004 四           four     four      None
0005 五           five     five      None
...               ...      ...       ...
c    踊        eagerly  eagerly      None
     錶          watch    watch      None
     锺  draw together     draw  together
     闺     small door    small      door
     鹭          heron    heron      None

[3053 rows x 3 columns]